##### Sessions: EV/EVSE

In [1]:
from scipy.optimize import fsolve
from pyomo.environ import *
import numpy as np
import numpy as np
import pandas as pd

np.random.seed(42)

m = 20  #  number of EVSEs
n = 10  #  number of time periods in the future
r = 0.5  # factor of Enexis grid connection

# --------------------------------------------------------------------------
# The array's for all EV's & SE together and the resulting minimum.
# EV = np.random.choice([EV_MPI, EV_MPI / 2, 3.33, 3.33 / 2], size=(m, n))
# SE = np.random.choice([SE_MPO], size=(m, n))
# EVSE = np.minimum(EV, SE)


SE_MPO = 7  #              EVSE max Power output for each time period
EV_MPI = 7  #              EV max Power input for each time period
EX_MPO = m * r * SE_MPO  # enexis max Power output for each time period (constant)
CAP = 70
K = 0.075
soc_l = 0.0; soc_h = 0.2
d_c_l = 0.4; d_c_h = 1.0

print_solver_outcome = False
print_EVSE_power = True

print_session_max_charge = True
print_desired_charge = True
print_realistic_charge = True

print_parking_time = True
print_energy_price = False

# tested with:
# solver = "ipopt" #  m 50 EVSE's, > 50 time periods
# solver = "mosek" #  m 50 EVSE's, > 50 time periods
# solver = "cplex"  # m 50 EVSE's, 18 time periods
solver = "glpk" #     m 50 EVSE's, > 50 time periods
# solver = "gurobi" # m xx EVSE's, 19 time periods - no license yet

alpha = 1.0  # EVSE efficiency
beta = 1.0  #  customer satisfaction
gamma = 1.0  # cost of energy

##### Charging Profile

In [2]:
def cv_pwr(t: float, pm: float, k: float) -> float:
    return pm * np.exp(-k * t)


def cv_eng(t2, t1, pm, k) -> float:
    return (-1 / k) * (cv_pwr(t2, pm, k) - cv_pwr(t1, pm, k))


def cv_pwr_avg(t2, t1, pm, k) -> float:
    return cv_eng(t2, t1, pm, k) / (t2 - t1)


def charge_profile(
    dur: float,
    soc: float,
    d_c: float,
    cap: float = CAP,
    ev_mpi: float = EV_MPI,
    k: float = K,
) -> list:
    """
    Calculate the charging profile for a given EV.

    Parameters
    ----------
    dur : float
        Duration of stay in hours.
    soc : float
        State of charge of the battery in %.
    d_c : float
        Desired charge of the battery in %.
    cap : float
        Capacity of the battery in kWh.
    ev_mpi : float
        Maximum power input of the EV in kW.
    k : float
        Charging curve constant.
        0.01-0.03 charge aggressively,
        0.05-0.1  prioritizing battery health and longevity
    cv : float
        Capacity at which the charging curve flattens in %.
    cc : float
        Capacity at which the charging curve starts in %.

    Returns
    -------
    list
        List of the charging profile in the format [time, power, charge].
    """
    # --------------------------------------------------------------------------
    # Calculate the charging profile
    # https://www.homechargingstations.com/ev-charging-time-calculator/
    # --------------------------------------------------------------------------
    # Calculate the charge required to reach the desired capacity
    cv = 0.8 * cap
    dc = d_c * cap

    # current charge
    uc0 = soc * cap
    up0 = ev_mpi
    ut0 = uc0 / up0

    # first part of charge < 80% of cap
    uc1 = max(min(dc, cv) - min(uc0, cv), 0)  #      charge [kWh] required
    up1 = 0 if uc1 == 0 else ev_mpi  #               power [kW] required
    ut1 = 0 if uc1 == 0 else uc1 / up1  #            time [h] required
    # constrainted by duration
    ct1 = min(dur, ut1)  #                           real time [h]  stay
    cp1 = up1  #                                     real power [kW]  stay
    cc1 = ct1 * cp1  #                               charge [kWh] during stay

    # second part of charge > 80% of cap
    uc2 = max(max(dc, cv) - max(uc0, cv), 0)  #      charge [kWh] required
    up2 = 0  #                                       initial power [kW] required
    ut2 = 0  #                                       initial time [h] required
    ct2 = 0  #                                       real time [h]  stay
    cp2 = 0  #                                       real power [kW]  stay
    cc2 = 0  #                                       charge [kWh] during stay

    # Define the function for the given equation with specific ta, pm and k
    def zero_for_E(t2, t1, pm, k, E) -> float:
        return cv_eng(t2, t1, pm, k) - E

    # determime time and power for 2nd part of charge CV
    if uc2 > 0:
        # Solve the equation numerically
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html
        solution = fsolve(func=zero_for_E, x0=ut1, args=(0, ev_mpi, k, uc2), xtol=1e-3)

        ut2 = solution[0]  #                         time [h] required during CV
        up2 = cv_pwr(ut2, ev_mpi, k)  #              power [kW] required during CV

        ct2 = max(0, min(dur - ut1, ut2))  #         real time [h] during CV
        cc2 = max(0, cv_eng(ct2, 0, ev_mpi, k))  #   charge [kWh] during CV
        cp2 = 0 if ct2 == 0 else up1  #              real power [kW] during CV
        # cp2 = cv_pwr(ct2, ev_mpi, k)  #            power [kW] required during CV

    # --------------------------------------------------------------------------
    e2 = 0 if ut2 == 0 else float(zero_for_E(t2=ut2, t1=0, pm=ev_mpi, k=k, E=uc2))

    ut3 = max(0, dur - (ut1 + ut2))

    return {
        "params": {
            "dur": dur,
            "soc": soc,
            "d_c": d_c,
            "cap": cap,
            "ev_mpi": ev_mpi,
            "k": k,
        },
        "phase0": {"c0": uc0, "t": ut0, "p": up0},
        "phase1": {"c1": uc1, "t": ut1, "p": up1},
        "real_1": {"c1": cc1, "t": ct1 + ut3, "p": cp1},
        "phase2": {"c2": uc2, "t": ut1 + ut2, "p": up2},
        "real_2": {"c2": cc2, "t": ct1 + ct2 + ut3, "p": cp2},
        # "phase3": {"c3": 0.0, "t": ut1 + ut2 + ut3, "p": 0.0},
        # "real_3": {"c3": 0.0, "t": ct1 + ct2 + ut3, "p": 0.0},
        "result": {
            "ufc": uc0 + uc1 + uc2 + e2,
            "cfc": uc0 + cc1 + cc2 + e2,
            "dc": dc,
            "rm": dc - (uc0 + uc1 + uc2 + e2),
        },
        "tslots": {"t1": ut1, "T1": ct1, "t2": ut2, "T2": ct2, "t3": ut3, "T3": ut3},
    }

$$
\int_{a}^{b} P_{\text{max}} \times e^{-k \times (t - t_0)} \, dt = 
-\frac{P_{\text{max}}}{k} \left[ e^{-k \times (b - t_0)} - e^{-k \times (a - t_0)} \right]

$$


##### TGC: Central Controller

In [3]:
# read data from ev
data = []
realistic_charge = []

for i in range(1, m + 1):
    cp = charge_profile(
        dur=np.random.uniform(1, 8),
        soc=np.random.uniform(soc_l, soc_h),
        d_c=np.random.uniform(d_c_l, d_c_h),
        cap=CAP,
        ev_mpi=EV_MPI,
        k=K,
    )
    ev = "ev" + str(i).zfill(2)
    data.append({"ev": ev, "t": cp["real_1"]["t"], "p": cp["real_1"]["p"]})
    data.append({"ev": ev, "t": cp["real_2"]["t"], "p": cp["real_2"]["p"]})
    realistic_charge.append(cp["result"]["cfc"])
    # data.append({"ev": ev, "t": cp["real_3"]["t"], "p": cp["real_3"]["p"]})
    print(ev, cp)
    if i==0:
        print(cp["params"])
        print(cp["phase0"])
        print(cp["phase1"])
        print(cp["real_1"])
        print(cp["phase2"])
        print(cp["real_2"])
        print(cp["result"])
        print(cp["tslots"])


# Create a DataFrame from the data
df = pd.DataFrame(data)

# Remove all records where 't' is 0
df = df.loc[df["t"] != 0]

# df_pivot = df.pivot(index='ev', columns='t', values='p')
df_pivot = df.pivot_table(index="ev", columns="t", values="p", aggfunc="sum")

# Replace NaN values in the last column with 0 or ev_mpi
df_pivot.iloc[:, -1] = df_pivot.iloc[:, -1].fillna(0)  # ev_mpi


# Replace NaN values in the other columns with the last non-NaN value
def fill_na_with_last_val(row):
    last_val = None
    for col in reversed(row.index):
        if pd.isna(row[col]):
            if last_val is not None:
                row[col] = last_val
        else:
            last_val = row[col]
    return row


df_pivot = df_pivot.apply(fill_na_with_last_val, axis=1)

# Keep only the first X columns or fewer if the DataFrame has fewer than X columns
df_pivot = df_pivot.iloc[:, : min(n, df_pivot.shape[1])]

# ------------------------------------------------------------------------------
# print("\n")
# print(df_pivot)

# Convert the DataFrame to a NumPy array
EVSE = df_pivot.values

if print_EVSE_power:
    print(f"\nEVSE:\n {EVSE}\n")


ev01 {'params': {'dur': 3.621780831931537, 'soc': 0.19014286128198324, 'd_c': 0.839196365086843, 'cap': 70, 'ev_mpi': 7, 'k': 0.075}, 'phase0': {'c0': 13.310000289738827, 't': 1.9014286128198326, 'p': 7}, 'phase1': {'c1': 42.689999710261176, 't': 6.098571387180168, 'p': 7}, 'real_1': {'c1': 25.35246582352076, 't': 3.621780831931537, 'p': 7}, 'phase2': {'c2': 2.7437455560790127, 't': 6.49641195324331, 'p': 6.79421902077194}, 'real_2': {'c2': 0, 't': 3.621780831931537, 'p': 0}, 'result': {'ufc': 58.74374638970747, 'cfc': 38.66246694688804, 'dc': 58.74374555607901, 'rm': -8.336284551546669e-07}, 'tslots': {'t1': 6.098571387180168, 'T1': 3.621780831931537, 't2': 0.3978405660631429, 'T2': 0, 't3': 0, 'T3': 0}}
ev02 {'params': {'dur': 5.190609389379256, 'soc': 0.031203728088487304, 'd_c': 0.49359671220172163, 'cap': 70, 'ev_mpi': 7, 'k': 0.075}, 'phase0': {'c0': 2.1842609661941115, 't': 0.3120372808848731, 'p': 7}, 'phase1': {'c1': 32.367508887926405, 't': 4.623929841132344, 'p': 7}, 'real_1

In [4]:
# Create the weights w[j] for the parking
absolute_times = df_pivot.columns.tolist()

pt = [j - i for i, j in zip(absolute_times[:-1], absolute_times[1:])]
pt.insert(0, absolute_times[0])

# pt = np.random.choice([40, 10, 20, 10, 10], size=(n))/60
if print_parking_time:
    print(f"\nparking time:\n {pt}\n")
w = pt / np.sum(pt)

# print(sum(pt[0:1]))



parking time:
 [1.2407196478065288, 0.16586563737086735, 0.5698717393868966, 0.3073745444097442, 0.08804846795997934, 0.7604163472795786, 0.4894844477179423, 0.4018342985632737, 0.057452325682398, 0.11142243334204238]



In [5]:
# desired charge for each session, max battery capacity = set to 70 kWh
# desired_charge = np.random.uniform(1, 20, size=m)
desired_charge = np.array(realistic_charge)
if print_desired_charge or True:
    print(f"\ndesired_charge:\n {desired_charge}\n")



desired_charge:
 [38.66246695 34.55176985 21.97256304 41.98374074 35.63664862 20.24621238
 32.24251388 17.92522425 36.38629885 29.95091733 30.73216691 61.95268906
 23.29348429 30.27600748 21.41552316 43.827547   36.37676676 58.1102711
 31.71668509 17.2363423 ]



In [6]:
# realistic charge for each session
# realistic_charge = np.minimum(session_max_charge, desired_charge)
realistic_charge = np.array(realistic_charge)
if print_realistic_charge:
    print(f"\nrealistic_charge:\n {realistic_charge}\n")


realistic_charge:
 [38.66246695 34.55176985 21.97256304 41.98374074 35.63664862 20.24621238
 32.24251388 17.92522425 36.38629885 29.95091733 30.73216691 61.95268906
 23.29348429 30.27600748 21.41552316 43.827547   36.37676676 58.1102711
 31.71668509 17.2363423 ]



In [7]:
# energy price in euro/kWh for each time period
# EnergyPrice = np.array([0.04, 0.08, 0.31, 0.28, 0.12])

EnergyPrice = np.linspace(0.1, 0.1, len(pt)) # np.random.uniform(-0.1, 0.4, size=n)
Energy_Cost = EnergyPrice * pt

if print_energy_price or True:
    print(f"\nenergy_price:\n {EnergyPrice}\n")


energy_price:
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]



In [8]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.I = RangeSet(1, m)  # set of EVSEs
model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)


# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# energy price per kWh for each time period
energy_price = {(j + 1): EnergyPrice[j] for j in range(len(EnergyPrice))}
model.energy_price = Param(model.J, initialize=energy_price)

# enexis max Power output for each time period (constant)
enexis_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.enexis = Param(model.J, initialize=enexis_kw_dv, default=EX_MPO)

# max Power EVSE session for each time period
session_max_kw = {
    (i + 1, j + 1): EVSE[i][j] for i in range(len(EVSE)) for j in range(len(EVSE[i]))
}
model.session = Param(model.I, model.J, initialize=session_max_kw)

# realistic charge for each session
realistic_charge_kwh = {
    (i + 1): realistic_charge[i] for i in range(len(realistic_charge))
}
model.realistic_charge = Param(model.I, initialize=realistic_charge_kwh)


# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.I, model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return (
        alpha
        * sum(
            w[j - 1] * (1 - sum(model.x[i, j] for i in model.I) / model.enexis[j])
            for j in model.J
        )
        + beta
        * sum(
            (
                1
                - sum(pt[j - 1] * model.x[i, j] for j in model.J)
                / model.realistic_charge[i]
            )
            for i in model.I
        )
        / m
        + (gamma / sum(Energy_Cost[j - 1] * model.enexis[j] for j in model.J))
        * sum(
            (Energy_Cost[j - 1] * sum(model.x[i, j] for i in model.I)) for j in model.J
        )
    )


model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power[i] <= enexis max power output for period j?
def ex_grid_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) <= model.enexis[j]


# is ev/se power <= session max power input for period j?
def session_constraint_rule(model, i, j):
    return model.x[i, j] <= model.session[i, j]


# is final charge <= realistic charge <= desired charge
def deschrg_constraint_rule(model, i):
    return sum(pt[j - 1] * model.x[i, j] for j in model.J) <= model.realistic_charge[i]


# the next line creates one constraint for each member of the set model.J
model.Ex_Grid_Constraint = Constraint(model.J, rule=ex_grid_constraint_rule)
model.DesChrg_Constraint = Constraint(model.I, rule=deschrg_constraint_rule)
model.Session_Constraint = Constraint(model.I, model.J, rule=session_constraint_rule)

# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory(solver)

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# outcome of solver model
if print_solver_outcome:
    print(f"\nsolver outcome:\n {tgc.pprint()}\n")


# print the power per EVSE per time period
def EVSE_print():
    for i in range(1, m + 1):
        print(
            f"EVSE {str(i).zfill(2)}: DC: {round(desired_charge[i-1], 2)} kWh\tRC: {round(realistic_charge[i-1], 2)} kWh\tFC: {round(sum(pt[x-1] * tgc.x[i, x].value for x in range(1, n+1)), 2)} kWh\tCS: {round(100*sum(pt[x-1] * tgc.x[i, x].value for x in range(1, n+1))/realistic_charge[i-1], 2)} %\tPwr: {[round(tgc.x[i, x].value, 2) for x in range(1, n+1)]} kW"
        )


def ENEXIS_print():
    for i in range(1, n + 1):
        print(
            f"PT: {pt[i-1]}\tPwr: {[sum(tgc.x[x, i].value for x in range(1, m+1))*100/CAP]} %"
        )


def TotalFC():
    ret = 0
    for x in range(1, n + 1):
        for i in range(1, m + 1):
            ret += pt[x - 1] * tgc.x[i, x].value
    return ret


def TotalRC():
    ret = 0
    for i in range(1, m + 1):
        ret += realistic_charge[i - 1]
    return ret


def TotCustSat_print():
    print(f"TCS: {round(TotalFC()/TotalRC(),2)*100} % ")


print("\nEVSE Results")
EVSE_print()

print("\nEnexis Utilization")
ENEXIS_print()

print("\nCustomer satisfaction")
TotCustSat_print()

# print(TotalFC())

# print the objective outcome
print("\n")
print("\nObjective Outcome = ", round(value(tgc.OBJ), 4))


EVSE Results
EVSE 01: DC: 38.66 kWh	RC: 38.66 kWh	FC: 3.43 kWh	CS: 8.86 %	Pwr: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 0.0, 0.0, 0.0] kW
EVSE 02: DC: 34.55 kWh	RC: 34.55 kWh	FC: 19.5 kWh	CS: 56.44 %	Pwr: [0.0, 0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0] kW
EVSE 03: DC: 21.97 kWh	RC: 21.97 kWh	FC: 9.85 kWh	CS: 44.81 %	Pwr: [7.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] kW
EVSE 04: DC: 41.98 kWh	RC: 41.98 kWh	FC: 3.99 kWh	CS: 9.52 %	Pwr: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.0, 7.0, 7.0] kW
EVSE 05: DC: 35.64 kWh	RC: 35.64 kWh	FC: 15.51 kWh	CS: 43.53 %	Pwr: [0.0, 0.0, 0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0] kW
EVSE 06: DC: 20.25 kWh	RC: 20.25 kWh	FC: 15.99 kWh	CS: 78.96 %	Pwr: [7.0, 7.0, 7.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] kW
EVSE 07: DC: 32.24 kWh	RC: 32.24 kWh	FC: 19.48 kWh	CS: 60.42 %	Pwr: [0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 0.0, 0.0] kW
EVSE 08: DC: 17.93 kWh	RC: 17.93 kWh	FC: 13.84 kWh	CS: 77.18 %	Pwr: [7.0, 7.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] kW
EVSE 09: DC: 36.